# Setup Enviroments 

### Download Request library

**NOTE**.  Only download this on initial start of platform

In [ ]:
# Only execute on initial start of platform
!pip install requests

### Define Spotify Developer Credentials

In [1]:
client_id = "cd1845f23d914228b14f6bed139ee594"
client_secret = ""

# Create an API Client Class for Spotify, `SpotifyAPI`

Rather than creating a function for Spotify Access Token that's a state-like item (change overtime), let's create a Class that can response to changes overtime.

This class authorize your access to Spotify API by: 
  1. Gain credentials by passing in Spotify Dev. ID and Secret Key
  1. Gain authorization by getting token

In [2]:
import requests
import base64
import datetime

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    
    token_URL = "https://accounts.spotify.com/api/token"
    
    
    
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    
    
    
    
    def get_client_credentials(self):
        """
        METHOD: Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if (client_id == None) or (client_secret == None):
            raise Exception("You must set a Client ID or a Client Secret with Spotify Developer Account.")
        
        # Convert Spotify Credentials from string to base64-bytes (decoded)
        client_creds = f"{client_id}:{client_secret}"                # type: String
        client_creds_b64 = base64.b64encode(client_creds.encode())   # type: Base64-Byte
        return client_creds_b64.decode()

    
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"             # Basic <base64 encoded client_id:client_secret>
        }
    
    
        
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
        
    
    
    
    
    def perform_auth(self):
        """
        METHOD: Authenticate to Spotify by acquiring token
        """
        token_URL = self.token_URL
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        # Get Token Response
        response = requests.post(token_URL, data=token_data, headers=token_headers)

        
        
        # Exit if Request is Invalid...
        if response.status_code not in range (200, 299):
            return False
        
        
        
        # Else, proceed...
        token_response_data = response.json()

        access_token = token_response_data['access_token']

        expires_in = token_response_data['expires_in']            # Unit: Seconds
        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now

        return True

# Calling API Client Class, `SpotifyAPI`

In [6]:
sp = SpotifyAPI(client_id, client_secret)

In [7]:
sp.perform_auth()

True

In [8]:
sp.access_token

'BQB45IRtAf87sfYIR7AiHs1bp38xcC9x7BMD9_cXKsrPCeqwN6BeP2d_tjrxwFkPmklxeXX5ibHqdD-gSuY'

# REFERENCES
  - https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow